In [1]:
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
sess = tf.InteractiveSession()
import os
import numpy as np

In [2]:
# Define functions

def conv2d(x, W, dy ,dx):
  return tf.nn.conv2d(x, W, strides=[1, dx, dy, 1], padding='SAME')

def max_pool_axb(x,a,b):
  return tf.nn.max_pool(x, ksize=[1, a, b, 1],
                        strides=[1, a, b, 1], padding='SAME')

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [3]:
# Define the parameters of the network

#-----------ACC 0.774 *** score 81----------------
nx_conv1=16      # filter_width (frequency)
my_conv1=4   # filter_height (time)
dnx_conv1=4    # stride_width
dmy_conv1=1    # stride_height

nx_pool1=4      # pooling width
my_pool1=1      # pooling height

channels_conv1=32    # channels num

nx_conv2=2
my_conv2=16
dnx_conv2=1
dmy_conv2=8

nx_pool2=2
my_pool2=1


channels_conv2=16

size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
num_neurons_fc1=64

print(size_input_fc1)

1024


In [4]:
# Define the parameters of the network



#-----------ACC 0.774 *** score 81----------------
# nx_conv1=16      # filter_width (frequency)
# my_conv1=4   # filter_height (time)
# dnx_conv1=4    # stride_width
# dmy_conv1=1    # stride_height

# nx_pool1=4      # pooling width
# my_pool1=1      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=2
# my_conv2=16
# dnx_conv2=1
# dmy_conv2=8

# nx_pool2=2
# my_pool2=1


# channels_conv2=16

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=64

# print(size_input_fc1)

#-----------ACC 0.65 *** score 74----------------
# nx_conv1=16      # filter_width
# my_conv1=4   # filter_height
# dnx_conv1=4    # stride_width
# dmy_conv1=1    # stride_height

# nx_pool1=4      # pooling width
# my_pool1=4      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=2
# my_conv2=16
# dnx_conv2=1
# dmy_conv2=8

# nx_pool2=2
# my_pool2=2


# channels_conv2=4

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=32

# print(size_input_fc1)

#--------------ACC 0.64  acore 69--------------
# nx_conv1=8      # filter_width
# my_conv1=8  # filter_height
# dnx_conv1=2    # stride_width
# dmy_conv1=2    # stride_height

# nx_pool1=2      # pooling width
# my_pool1=2      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=4
# my_conv2=4
# dnx_conv2=2
# dmy_conv2=2

# nx_pool2=2
# my_pool2=2


# channels_conv2=4

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=32

# print(size_input_fc1)



In [36]:
x = tf.placeholder("float", shape=[None, 128*128])
y_ = tf.placeholder("float", shape=[None, 2])


x_image = tf.reshape(x, [-1,128,128,1])

W_conv1 = weight_variable([nx_conv1,my_conv1,1,channels_conv1])
b_conv1 = bias_variable([channels_conv1])


h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, dnx_conv1, dmy_conv1 ) + b_conv1)
h_pool1 = max_pool_axb(h_conv1, nx_pool1, my_pool1)


W_conv2 = weight_variable([nx_conv2, my_conv2, channels_conv1, channels_conv2])
b_conv2 = bias_variable([channels_conv2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, dnx_conv2, dmy_conv2 ) + b_conv2)
h_pool2 = max_pool_axb(h_conv2, nx_pool2, my_pool2)



W_fc1 = weight_variable([size_input_fc1, num_neurons_fc1])
b_fc1 = bias_variable([num_neurons_fc1])

h_pool2_flat = tf.reshape(h_pool2, [-1, size_input_fc1])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


W_fc2 = weight_variable([num_neurons_fc1, 2])
b_fc2 = bias_variable([2])



keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)



cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


In [37]:
testData=sio.loadmat('C:/OSU/SongMat/linear/Test/532522 SakiZ - osu!memories.mat')
inputTest=testData['input'].reshape(-1,128*128)
targetTest=testData['target'].reshape(-1,4)
targetTest2=np.zeros([targetTest.shape[0],2])
targetTest2[:,0]=targetTest[:,0]+targetTest[:,1]+targetTest[:,2]
targetTest2[:,1]=targetTest[:,3]
NTest=targetTest2.shape[0]

In [38]:


# merged = tf.summary.merge_all()
# train_writer = tf.summary.FileWriter('/train',sess.graph)
# test_writer = tf.summary.FileWriter('/test')
# tf.global_variables_initializer().run()

sess.run(tf.global_variables_initializer())


In [39]:
saver = tf.train.Saver()


In [40]:
step_size=200
 
dataList=os.listdir('C:/OSU/SongMat/linear/Train/')
acc=np.zeros([100,1])

for n in range(int(np.size(dataList)/1)-15):
    inputOsu=np.empty([0,128*128]);
    outputOsu=np.empty([0,4]);
    for k in range(n,n+15):
        
        data=sio.loadmat('C:/OSU/SongMat/linear/Train/'+dataList[k])
        inputOsu=np.append(inputOsu,data['input1'].reshape(-1,128*128),axis=0)
        outputOsu=np.append(outputOsu,data['target1'].reshape(-1,4),axis=0)
               
        outputOsu2=np.zeros([outputOsu.shape[0],2]);
        outputOsu2[:,0]=outputOsu[:,0]+outputOsu[:,1]+outputOsu[:,2]
        outputOsu2[:,1]=outputOsu[:,3]
        
    N=inputOsu.shape[0]
    print(N)
    #shuffle the order
    order = np.arange(N-3000)
    np.random.shuffle(order)
    
    for i in range(N-3000):
        if i%step_size == 0:
            
            train_accuracy = accuracy.eval(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: outputOsu2[order[i:i+step_size],:], keep_prob: 1.0})
#             print("step %d, training accuracy %g" %(i, train_accuracy))
            train_step.run(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: outputOsu2[order[i:i+step_size],:], keep_prob: 0.7})
    
    
    # Test accuracy
    orderAcc = np.arange(NTest)
    np.random.shuffle(orderAcc)
    accTest=accuracy.eval(feed_dict={x: inputTest[orderAcc[0:500],:], y_: targetTest2[orderAcc[0:500],:], keep_prob: 1.0});
    print("test accuracy %g"%accTest)
    print(y_conv.eval(feed_dict={x: inputTest[orderAcc[0:10],:], keep_prob: 1.0}))
    acc[n]=accTest
    print(n)
    
    if accTest>0.9:
        break
        
        

21236
test accuracy 0.63
[[ 0.65631771  0.34368226]
 [ 0.59877497  0.40122503]
 [ 0.53473914  0.46526089]
 [ 0.50340855  0.49659148]
 [ 0.53511524  0.4648847 ]
 [ 0.58571184  0.41428822]
 [ 0.64957303  0.350427  ]
 [ 0.59038466  0.40961537]
 [ 0.4164032   0.58359677]
 [ 0.50782645  0.49217352]]
0
24198
test accuracy 0.726
[[ 0.71863723  0.28136274]
 [ 0.60262394  0.39737606]
 [ 0.3386769   0.66132313]
 [ 0.55293739  0.44706258]
 [ 0.28374264  0.71625733]
 [ 0.44665954  0.55334044]
 [ 0.36247736  0.63752264]
 [ 0.65566629  0.34433374]
 [ 0.62600756  0.37399247]
 [ 0.06148507  0.93851495]]
1
23732
test accuracy 0.7
[[ 0.60592306  0.39407691]
 [ 0.54386717  0.45613283]
 [ 0.45509288  0.54490709]
 [ 0.64942765  0.35057226]
 [ 0.46139988  0.53860009]
 [ 0.63402969  0.36597028]
 [ 0.58883327  0.41116679]
 [ 0.70586425  0.29413569]
 [ 0.77467102  0.22532901]
 [ 0.60289937  0.39710066]]
2
22078
test accuracy 0.766
[[ 0.6409055   0.3590945 ]
 [ 0.45920199  0.54079801]
 [ 0.53799784  0.46200207]

 [ 0.51155794  0.488442  ]]
27
20773
test accuracy 0.544
[[ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]
 [ 0.49290058  0.50709951]]
28
21041
test accuracy 0.492
[[ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]
 [ 0.50593835  0.49406162]]
29
17119
test accuracy 0.514
[[ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]
 [ 0.50219309  0.49780691]]
30
18041
test accuracy 0.522
[[ 0.49795485  0.50204521]
 [ 0.49795485  0

 [ 0.5206582   0.47934183]]
55
24021
test accuracy 0.438
[[ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]
 [ 0.51843941  0.48156065]]
56
24015
test accuracy 0.5
[[ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]
 [ 0.51815504  0.4818449 ]]
57
25927
test accuracy 0.476
[[ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]
 [ 0.52260232  0.47739765]]
58


KeyboardInterrupt: 

In [ ]:
save_path = saver.save(sess, "D:/OSU/results/save/",global_step=n)

In [41]:
print(acc)

[[ 0.63      ]
 [ 0.72600001]
 [ 0.69999999]
 [ 0.76600003]
 [ 0.70600003]
 [ 0.72800004]
 [ 0.68400002]
 [ 0.68000007]
 [ 0.69800001]
 [ 0.71799999]
 [ 0.68399996]
 [ 0.69599998]
 [ 0.68599993]
 [ 0.67600006]
 [ 0.68000001]
 [ 0.67799997]
 [ 0.66999996]
 [ 0.68800002]
 [ 0.69400001]
 [ 0.67999995]
 [ 0.71400005]
 [ 0.64400005]
 [ 0.65200001]
 [ 0.65199995]
 [ 0.66400003]
 [ 0.46600002]
 [ 0.484     ]
 [ 0.46599996]
 [ 0.54399997]
 [ 0.49199998]
 [ 0.514     ]
 [ 0.52200001]
 [ 0.484     ]
 [ 0.51999998]
 [ 0.46799999]
 [ 0.47600001]
 [ 0.46000001]
 [ 0.51799995]
 [ 0.5       ]
 [ 0.46799999]
 [ 0.472     ]
 [ 0.52199996]
 [ 0.44199997]
 [ 0.47      ]
 [ 0.44400004]
 [ 0.46600002]
 [ 0.46200001]
 [ 0.47999996]
 [ 0.47600001]
 [ 0.50200003]
 [ 0.47      ]
 [ 0.46999997]
 [ 0.48800001]
 [ 0.46200004]
 [ 0.458     ]
 [ 0.44599998]
 [ 0.43799999]
 [ 0.49999997]
 [ 0.47600001]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

In [ ]:
print(y_conv.eval(feed_dict={x: inputOsu[200:210:1,:], keep_prob: 1.0}))
print ("test accuracy %g"%accuracy.eval(feed_dict={
  x: inputOsu[-500:,:], y_: outputOsu2[-500:,:], keep_prob: 1.0}))

In [ ]:
# Save the output of new maps 

dataList=os.listdir('D:/OSU/SongMat/Create/')
# inputOsuNew=np.empty([0,128*128]);
for i in range(np.size(dataList)):
    if not dataList[i][0]=='y':
        data=sio.loadmat('D:/OSU/SongMat/Create/'+dataList[i])
        print(data['osuFolder'])
        
        inputOsuNew=data['input'].reshape(-1,128*128)
        outputOsuNew=data['target']
        outputOsu2New=np.zeros([outputOsuNew.shape[0],2]);
        outputOsu2New[:,0]=outputOsuNew[:,0]+outputOsuNew[:,1]+outputOsuNew[:,2]
        outputOsu2New[:,1]=outputOsuNew[:,3]
        
        y_result=np.empty([0,2])
        
        step_size=500
        cursor=0
        print ("test accuracy %g"%accuracy.eval(feed_dict={x: inputOsuNew[-500:,:], y_: outputOsu2New[-500:,:], keep_prob: 1.0}))
        while cursor<inputOsuNew.shape[0]:
            if inputOsuNew[cursor:,:].shape[0]>step_size:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            else:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            cursor+=step_size
            
        data_save={'y2':y_result}
        sio.savemat('D:/OSU/SongMat/Create/y2_'+dataList[i],data_save)


In [ ]:
data_save={'y':y_result}
sio.savemat('D:/OSU/results/innocence/y_innocence.mat',data_save)

In [ ]:
checkpoint_dir='D:/OSU/results/innocence/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
saver2 = tf.train.Saver()

In [ ]:
checkpoint_dir='/Users/dongqihan/Desktop/FrOstNova/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver2.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess_tmp = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess_tmp.run(c))

In [ ]:
data_test=sio.loadmat('/Users/dongqihan/Desktop/FrOstNova/granat.mat')
inputOsu_test=data_test['input'].reshape(-1,128*128)
y_result_test=y_conv.eval(feed_dict={x: inputOsu_test, keep_prob: 1.0})
data={'y':y_result_test}
sio.savemat('/Users/dongqihan/Desktop/FrOstNova/y_memory2granat.mat',data)